In [1]:
# Install dependencies
!pip install matching

In [1]:
import numpy as np

In [98]:


        
def runExperiment(numUsers, userDim, steps, makeUsers, selectInteract, combinationFunc, config, logger=None):
    users = makeUsers(numUsers, userDim, config)
    print(np.mean(users, axis=0))
    print(users)
    for t in range(steps):
        prev = users
        users = combinationFunc(users, selectInteract(users, config), config)
        if not logger is None:
            logger(prev, users, t)
    return users
            

/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [99]:
class ConfigHelper(object):
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)


/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [100]:
# Averaging users
# Users start out by some distribution, then simply shift towards the other user they see. This ends them up at the average of all users
def randomSelectInteract(users, config):
    numUsers, userDim = users.shape
    randomPairing = np.arange(numUsers)
    np.random.shuffle(randomPairing)
    return users[randomPairing]
def normalDistrUsers(numUsers, userDim, config):
    return np.random.normal(0, 1, [numUsers, userDim])
def shiftTowards(users1, users2, config):
    return users1*(1-config.weightShifted)+users2*config.weightShifted


config = ConfigHelper(weightShifted=0.4)

runExperiment(10, 1, 1000, normalDistrUsers, randomSelectInteract, shiftTowards, config)

[0.14818084]
[[ 0.30846817]
 [ 0.75416702]
 [ 1.93256506]
 [-1.99272063]
 [ 0.49814381]
 [-0.52542622]
 [ 1.36383108]
 [-0.31811733]
 [-0.62373674]
 [ 0.08463418]]


/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.14818084],
       [0.14818084],
       [0.14818084],
       [0.14818084],
       [0.14818084],
       [0.14818084],
       [0.14818084],
       [0.14818084],
       [0.14818084],
       [0.14818084]])

In [101]:
# Q: How could you have a continuous representation but allow for meaningful weirdness? One option would be to show people to others based on how similar they are.
# One way: figure out some "optimal pairing" thing that pairs people to the person that is the most similar, instead of uniform
# This is just a stable roomates problem

# Makes a dictionary with preferences in sorted order based on dot product (highest dot product first, lowest dot product last)
def convertToRoomatePreferences(users):
    numUsers, userDim = users.shape
    userPreferences = {}
    for i in range(numUsers):
        indicesExceptMe = np.arange(numUsers)!=i
        otherUsers = users[indicesExceptMe] # everyone except this user
        otherUserIndices = np.arange(numUsers)[indicesExceptMe]
        user = users[i]
        # sort other users by preference (high dot product = high preference)
        # we need to look up the indices, since otherwise due to argmax they'd be shifted since we don't have this user
        preferences = otherUserIndices[np.argsort(-(user*otherUsers).sum(axis=1))]
        userPreferences[i] = list(preferences)
    return userPreferences


import matching
from matching.games import StableRoommates

# Uses stable roomates to find matching of users that are closest
def optimalSelectInteract(users, config):
    numUsers, userDim = users.shape
    stable = StableRoommates.create_from_dictionary(convertToRoomatePreferences(users))
    matches = stable.solve()
    # Need to lookup player object by name
    playerMap = {}
    for p in matches._data.keys():
        playerMap[p.name] = p
    # return users indexed by the found matching
    return users[list([matches[playerMap[i]].name for i in range(numUsers)])]


config = ConfigHelper(weightShifted=0.4)
# This results in the pairings converging together because the same pairing will be chosen every time
runExperiment(10, 1, 1000, normalDistrUsers, optimalSelectInteract, shiftTowards, config)



/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[-0.39715381]
[[ 0.86757424]
 [-1.34053453]
 [-1.74426109]
 [-0.87778229]
 [ 0.37154911]
 [ 0.2640467 ]
 [-0.50692817]
 [-1.22418065]
 [ 0.08962076]
 [ 0.1293578 ]]


array([[ 0.61956168],
       [-1.54239781],
       [-1.54239781],
       [-1.05098147],
       [ 0.61956168],
       [ 0.19670225],
       [-0.2086537 ],
       [-1.05098147],
       [-0.2086537 ],
       [ 0.19670225]])

In [102]:

def shiftTowardsIfCloseAwayIfFar(users1, users2, config):
    resIfSameDir = users1*(1-config.weightShifted)+users2*config.weightShifted
    # we want x s.t. users1 + x = resIfSameDir
    # x = resIfSameDir - users1
    shiftedAmountIfSameDir = resIfSameDir - users1
    # Then for "pushing away" we can just go in the opposite direction
    resIfOppositeDir = users1 - shiftedAmountIfSameDir
    dotProds = users1*users2
    # set to zero if it doesn't apply, then we can just add them
    resIfSameDir[dotProds<=0] = 0
    resIfOppositeDir[dotProds>0] = 0
    #print(np.c_[resIfSameDir, resIfOppositeDir])
    #print(resIfSameDir + resIfOppositeDir)
    return np.clip(resIfSameDir + resIfOppositeDir, -1, 1)


config = ConfigHelper(weightShifted=0.9)

runExperiment(10, 10, 1000, normalDistrUsers, randomSelectInteract, shiftTowardsIfCloseAwayIfFar, config)

[-0.22088506 -0.1273602   0.08131599 -0.64738389 -0.50843688 -0.1421048
 -0.08157529  0.10630013 -0.27485608 -0.1736708 ]
[[-0.43073023 -2.28292706 -0.53978495 -1.05708292 -0.8074776   0.50662036
   0.17883036  0.1154936   1.08382272 -0.43132586]
 [-0.72029372 -0.06994506  0.10250412 -1.33231163 -0.6884045   0.80321782
   0.50291919 -1.18228864  1.36249314 -1.26275549]
 [-1.47563916 -0.67321791 -1.45195823 -0.85776078 -0.88070139  1.02480775
  -0.85173474  0.31918041 -2.87448093  0.56913848]
 [-0.25781305  0.92530523  1.51005418 -1.35881134 -0.37004913 -0.55573237
  -0.13601119  0.87024857 -1.201944    0.93334372]
 [-0.19697714  0.39762028  2.36229487 -1.50326682 -0.47912941  0.2587594
  -0.07575216  1.59590749  0.92396776 -1.36672784]
 [ 1.20159388  1.96191541 -0.95997735 -0.34964664  0.04339783 -0.05573896
  -0.2501887  -1.04176351 -1.39574248 -0.06389641]
 [ 0.9660747   0.04782517 -0.34703756  0.36007432 -0.02544113  0.30943128
  -0.66023762  1.16040703  0.78681568 -0.7197969 ]
 [-1

/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[-1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.],
       [-1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.],
       [-1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,  1.],
       [-1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.],
       [ 1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1.],
       [ 1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.],
       [-1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1.],
       [-1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1.],
       [-1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.]])

In [132]:
def shiftTowardsButClamped(users1, users2, config):
    res = users1*(1-config.weightShifted)+users2*config.weightShifted
    return np.clip(res, -1, 1)

def logger(users, prev, t):
    if np.all(users == prev):
        print(f"unchanged at time {t}")
    #else:
    #    print(f"changed at time {t}")

config = ConfigHelper(weightShifted=0.1)
runExperiment(10, 10, 1000, normalDistrUsers, randomSelectInteract, shiftTowardsIfCloseAwayIfFar, config, logger)


[ 0.27152804  0.72893838  0.10573215 -0.45859073  0.87527062 -0.2540146
 -0.19006078 -0.25665657 -0.33421095  0.15361615]
[[-1.2945306   1.14827312  1.29914809 -0.33940147  1.74743444 -0.99094196
   0.83368797 -0.01815751 -0.38876099  0.29847462]
 [ 0.40298715 -1.20627911  0.16013798  0.31215455 -1.76356802 -0.2536598
  -0.41421135  0.82518797 -0.0319578   0.40079596]
 [ 0.20615118  0.7279826  -1.85814164 -1.29313476  1.23174396  0.95174293
  -0.29483776 -0.50780534 -0.54616996  1.0242515 ]
 [ 0.67942452  1.44600791  1.32213919  0.19925283  0.40573835 -1.15598062
   0.66668074 -1.39984841 -0.158001   -0.55255161]
 [ 0.52908692  0.98513958  0.93098052 -0.41790183  0.50650476 -0.68814262
  -0.09705386 -0.37646123 -1.20030118  0.41382169]
 [-0.7692092   0.72397465  0.49792252 -2.38270946  0.92514703  1.31462516
  -1.43019645 -0.30960936 -1.57913111  0.27877566]
 [ 1.16660844  0.80122607  0.48981421 -0.67837659  2.26190271 -0.90445735
  -1.233935    0.21992395 -0.8207838   0.30327048]
 [ 0

/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[-1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.],
       [ 1., -1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.],
       [ 1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.],
       [ 1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.],
       [ 1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1.],
       [-1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.,  1.],
       [ 1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.],
       [ 1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.],
       [ 1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1.],
       [-1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.]])

/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [135]:

users = np.random.normal(0, 1, [4, 4])

/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [126]:

# From https://www.nature.com/articles/s41599-019-0343-5
# a relationship between two traits is a score: 1 if they are compatible, -1 if they are incompatible
# There are 100 agents, each with an individual "cultural repertoire", having a subset of traits from the universe
# They start with no traits, but will slowly either acquire traits from the universe of traits, or through copying them from other agents
# Interactions look like this:
# 1. For each agent (the receiver), sample one other agent as a "cultural model"
#   Randomly select one of the traits, i, in its repertoire for display to the receiver
#   The receiver copies the trait with a probability determined by the average compatability score s of the trait with the receiver's current repertoire
#     If the receiver has no traits, s = 0
#   Probability of copying is determined by 1/(1+exp(-10*s))
#  10 was arbitrarily chosen, but values below 5 give the score a small influence and results weren't sensitive to things above that
# 2. Each agent invents a new trait with probability 0.001 (invents means "selects a trait from the universe and obtains it if it doesn't already have it")
# 3. Each agent dies with probability 0.01 (though 0.0025 provides more stability) and is replaced by a new naive agent

# one option: just have a pre-specified matrix of trait compatibilties, and each agent has a default affinity for every trait


numTraits = 20
traitMatrix = np.random.normal(0, 1, [numTraits, numTraits])
config = ConfigHelper(numTraits=numTraits, traitMatrix=traitMatrix)

def interactUsers(users1, users2, config):
    resUsers = []
    numUsers, userDim = users1.shape
    for i in range(numUsers):
        preferences = (users[0]*(users[])).sum(axis=1)
        np.argsort(preferences)
        

def traitUsers(numUsers, userDim, config):
    return np.zeros([numUsers, userDim])



/home/azureuser/miniconda3/envs/sandbox1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SyntaxError: invalid syntax (<ipython-input-126-f09ea39a7ae2>, line 26)

In [84]:
users = np.random.random([4, 6])
users

array([[0.86701282, 0.84294248, 0.92500557, 0.59653969, 0.22788833,
        0.550036  ],
       [0.91282008, 0.27975259, 0.24810738, 0.08824584, 0.75125026,
        0.0065273 ],
       [0.73913925, 0.08258533, 0.90970609, 0.85146818, 0.69061477,
        0.54597442],
       [0.56588378, 0.13174201, 0.25497804, 0.36703337, 0.32849429,
        0.33089205]])

In [101]:
np.argsort((users[0]*(users)).sum(axis=1))


array([3, 1, 2, 0])

array([3.02823085, 1.48417632, 2.51756428, 1.31334806])